In [1]:
import pandas as pd
import gzip
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

 # A la empresa le interesa el rastreo de lo que marketing considera los productos y lo que ventas considera los mejores clientes.

## Mejores Productos
- product_id = {20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032} (diez productos)

## Mejores Clientes
- customer_id = {10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013} (doce clientes)

Total de <producto, cliente> a predecir = 10 * 12 = 120

# Objetivo
- Es el 01-enero-2020 a las 00:01 y disponibilizamos las ventas del periodo 2021912.
- El 02-enero a las 18:00 nos deben entregar:
  - El primer forecast de ventas para cada producto que se harán durante el mes 202002, de forma que nuestras plantas puedan fabricarlos durante el mes de 202001.
  - El segundo forecast es las ventas esperadas en 202002, para los 120 pares de <mejores_clientes, mejores_productos>.

In [2]:
####################################################
############# Setear segun cada maquina ############
os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")
####################################################

In [3]:
arch_sellout = "tb_sellout_02.txt.gz"
arch_maestro_prod = "tb_productos_02.txt"
arch_exogenadas = "exogenas.csv"

## Maestro de productos

In [4]:
df_product = pd.read_csv(arch_maestro_prod,sep="\t")

In [5]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cat1        1262 non-null   object
 1   cat2        1262 non-null   object
 2   cat3        1262 non-null   object
 3   brand       1262 non-null   object
 4   sku_size    1262 non-null   int64 
 5   product_id  1262 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 59.3+ KB


In [6]:
# Por las dudas, eliminamos duplicados
print(len(df_product))
df_product.drop_duplicates(inplace=True)
print(len(df_product))

1262
1253


In [7]:
print(len(df_product.product_id.unique()))
print(len(df_product))

1251
1253


In [8]:
# Revisamos cuales son los que tienen mas de una descripcion
df_product.groupby("product_id").filter(lambda x: len(x) > 1)

,cat1,cat2,cat3,brand,sku_size,product_id
915,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,350,20623
917,PC,CABELLO,ACONDICIONADOR,SHAMPOO3,350,20230
981,PC,CABELLO,SHAMPOO,SHAMPOO3,350,20623
983,PC,CABELLO,SHAMPOO,SHAMPOO3,350,20230


In [9]:
# Eliminamos los que son acondicionador
df_product = df_product[~((df_product.product_id.isin([20623,20230])) & (df_product.cat3=="ACONDICIONADOR"))]
print(len(df_product.product_id.unique()))
print(len(df_product))

1251
1251


In [10]:
df_product.cat1.value_counts(dropna=False)

PC       721
HC       292
FOODS    225
REF       13
Name: cat1, dtype: int64

In [11]:
df_product[df_product.cat1=="REF"]

,cat1,cat2,cat3,brand,sku_size,product_id
673,REF,TE,Frutas,TWININGS,20,21271
674,REF,TE,Hierbas,TWININGS,20,21218
675,REF,TE,Especial,TWININGS,10,21257
676,REF,TE,Especial,TWININGS,20,21181
677,REF,TE,Especial,TWININGS,10,21255
678,REF,TE,Especial,TWININGS,20,21170
679,REF,TE,Hierbas,TWININGS,20,21202
680,REF,TE,Frutas,TWININGS,20,21264
681,REF,TE,Verde,TWININGS,20,21192
682,REF,TE,Azul,TWININGS,20,20982


--> los reasignaremos a FOODS

In [12]:
df_product.cat1 = np.where(df_product.cat1=="REF","FOODS",df_product.cat1)

In [13]:
df_product.cat1.value_counts(dropna=False)

PC       721
HC       292
FOODS    238
Name: cat1, dtype: int64

In [14]:
df_product.cat2.value_counts(dropna=False)

CABELLO                352
DEOS                   168
SOPAS Y CALDOS         133
PIEL2                  114
ROPA LAVADO             99
HOGAR                   77
PIEL1                   73
ADEREZOS                69
VAJILLA                 47
PROFESIONAL             34
ROPA ACONDICIONADOR     33
OTROS                   23
DENTAL                  14
TE                      13
ROPA MANCHAS             2
Name: cat2, dtype: int64

In [15]:
df_product.cat3.value_counts(dropna=False)

SHAMPOO           146
ACONDICIONADOR    116
Aero              114
Liquido            63
Polvo              48
                 ... 
Pure de Tomate      1
Jabon Hotel         1
Jabon Barba         1
Chimichurri         1
Aji Picante         1
Name: cat3, Length: 99, dtype: int64

## Sellout

In [16]:
# Abrir el archivo .gz y cargarlo en un DataFrame
with gzip.open(arch_sellout, 'rt') as archivo:
    # Leer el archivo línea por línea
    df_sellout = pd.read_csv(archivo,sep="\t")

In [17]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [18]:
# Por las dudas, eliminamos duplicados
print(len(df_sellout))
df_sellout.drop_duplicates(inplace=True)
print(len(df_sellout))

2945818
2945818


--> sin duplicados

In [19]:
df_sellout.periodo.unique()                

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912], dtype=int64)

In [20]:
len(df_sellout.product_id.unique())          

1233

--> algunos productos no van a tener descripción

In [21]:
len(df_sellout.customer_id.unique())            

597

In [22]:
df_sellout.plan_precios_cuidados.unique()          

array([0, 1], dtype=int64)

In [23]:
df_sellout['periodo_fecha'] = pd.to_datetime(df_sellout['periodo'], format='%Y%m')

In [24]:
df_sellout.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01


In [25]:
# Como control, sumo tns
print("Toneladas Total Control:", round(sum(df_sellout.tn),0))

Toneladas Total Control: 1324989.0


In [26]:
print(len(df_sellout))
df_sellout = pd.merge(df_sellout, df_product, on='product_id', how='left')
print(len(df_sellout))
df_sellout.head()

2945818
2945818


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,cat1,cat2,cat3,brand,sku_size
0,201701,10234,20524,0,2,0.05300,0.05300,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0
1,201701,10032,20524,0,1,0.13628,0.13628,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0
2,201701,10217,20524,0,1,0.03028,0.03028,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0
3,201701,10125,20524,0,1,0.02271,0.02271,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0
4,201701,10012,20524,0,11,1.54452,1.54452,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0


In [27]:
df_sellout.cat1 = np.where(df_sellout.cat1.isna(),"desconocida",df_sellout.cat1)
df_sellout.cat2 = np.where(df_sellout.cat2.isna(),"desconocida",df_sellout.cat2)
df_sellout.cat3 = np.where(df_sellout.cat3.isna(),"desconocida",df_sellout.cat3)
df_sellout.brand = np.where(df_sellout.brand.isna(),"desconocida",df_sellout.brand)
df_sellout.sku_size = np.where(df_sellout.sku_size.isna(),0,df_sellout.sku_size)

## Exogenas

In [28]:
df_exogenas = pd.read_csv("emp3_exogenas.csv")
df_exogenas.periodo_fecha = pd.to_datetime(df_exogenas.periodo_fecha)

df_exogenas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periodo_fecha              48 non-null     datetime64[ns]
 1   temp_media                 48 non-null     float64       
 2   temp_max_media             48 non-null     float64       
 3   temp_min_media             48 non-null     float64       
 4   IPC                        48 non-null     float64       
 5   promedio_mens_dolar_venta  48 non-null     float64       
 6   catastrofe                 48 non-null     bool          
dtypes: bool(1), datetime64[ns](1), float64(5)
memory usage: 2.4 KB


## Generación de datasets base

**El primer forecast de ventas para cada producto que se harán durante el mes 202002, de forma que nuestras plantas puedan fabricarlos durante el mes de 202001.**

In [29]:
df_base_prod = df_sellout.groupby(['periodo','periodo_fecha','product_id']).agg({'tn': 'sum', 'cust_request_qty':'sum','cust_request_tn':'sum','plan_precios_cuidados':'max','cat1': 'max', 'cat2': 'max', 'cat3': 'max', 'sku_size':'max'}).reset_index()
print("Toneladas Total Control:", round(sum(df_base_prod.tn),0))

Toneladas Total Control: 1324989.0


In [30]:
# generamos nuevos tn y cust_request_tn con normalizacion

# Normalizacion MinMaxScaler lineal (0-1)
scaler = MinMaxScaler()
df_base_prod[['tn_01', 'cust_request_tn_01']] = scaler.fit_transform(df_base_prod[['tn', 'cust_request_tn']])

# Normalizacion standard con media y desvio
scaler = StandardScaler()
df_base_prod[['tn_md', 'cust_request_tn_md']] = scaler.fit_transform(df_base_prod[['tn', 'cust_request_tn']])

In [31]:
# Lista con el nuevo orden de las columnas
new_order = ['periodo','periodo_fecha','product_id','tn','tn_01','tn_md','cust_request_qty','cust_request_tn',
             'cust_request_tn_01', 'cust_request_tn_md', 'plan_precios_cuidados', 'cat1','cat2','cat3','sku_size']

# Reordena las columnas del DataFrame
df_base_prod = df_base_prod[new_order]

In [32]:
df_base_prod.head()

,periodo,periodo_fecha,product_id,tn,tn_01,tn_md,cust_request_qty,cust_request_tn,cust_request_tn_01,cust_request_tn_md,plan_precios_cuidados,cat1,cat2,cat3,sku_size
0,201701,2017-01-01,20001,934.77222,0.407273,8.128089,479,937.72717,0.386897,7.876399,0,HC,ROPA LAVADO,Liquido,3000.0
1,201701,2017-01-01,20002,550.15707,0.239699,4.624822,391,555.18654,0.229065,4.507684,0,HC,ROPA LAVADO,Liquido,3000.0
2,201701,2017-01-01,20003,1063.45835,0.463340,9.300226,438,1067.81543,0.440571,9.021977,0,FOODS,ADEREZOS,Mayonesa,475.0
3,201701,2017-01-01,20004,555.91614,0.242208,4.677278,339,569.37394,0.234918,4.632621,0,FOODS,ADEREZOS,Mayonesa,240.0
4,201701,2017-01-01,20005,494.27011,0.215349,4.115776,249,494.60084,0.204068,3.974157,0,FOODS,ADEREZOS,Mayonesa,120.0


In [33]:
# Le agregamos las exogenas
print(len(df_base_prod))
df_base_prod = pd.merge(df_base_prod,df_exogenas,on="periodo_fecha",how="left")
print(len(df_base_prod))

31243
31243


In [34]:
df_base_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31243 entries, 0 to 31242
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periodo                    31243 non-null  int64         
 1   periodo_fecha              31243 non-null  datetime64[ns]
 2   product_id                 31243 non-null  int64         
 3   tn                         31243 non-null  float64       
 4   tn_01                      31243 non-null  float64       
 5   tn_md                      31243 non-null  float64       
 6   cust_request_qty           31243 non-null  int64         
 7   cust_request_tn            31243 non-null  float64       
 8   cust_request_tn_01         31243 non-null  float64       
 9   cust_request_tn_md         31243 non-null  float64       
 10  plan_precios_cuidados      31243 non-null  int64         
 11  cat1                       31243 non-null  object        
 12  cat2

In [35]:
# Exportar el DataFrame a un archivo CSV
df_base_prod.to_csv("emp3_sellout_base_period_product.csv", index=False)

**El segundo forecast es las ventas esperadas en 202002, para los 120 pares de <mejores_clientes, mejores_productos>.**

In [36]:
# Filtrar productos y clientes
products_to_filter = [20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032]
customers_to_filter = [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013]
filtered_df = df_sellout[(df_sellout['product_id'].isin(products_to_filter))]
filtered_df = filtered_df[(filtered_df['customer_id'].isin(customers_to_filter))]

In [37]:
# generamos nuevos tn y cust_request_tn con normalizacion

# Normalizacion MinMaxScaler lineal (0-1)
scaler = MinMaxScaler()
filtered_df[['tn_01', 'cust_request_tn_01']] = scaler.fit_transform(filtered_df[['tn', 'cust_request_tn']])

# Normalizacion standard con media y desvio
scaler = StandardScaler()
filtered_df[['tn_md', 'cust_request_tn_md']] = scaler.fit_transform(filtered_df[['tn', 'cust_request_tn']])

In [38]:
# Lista con el nuevo orden de las columnas
new_order = ['periodo','periodo_fecha', 'product_id', 'customer_id', 'tn','tn_01','tn_md','cust_request_qty','cust_request_tn',
             'cust_request_tn_01', 'cust_request_tn_md', 'plan_precios_cuidados', 'cat1','cat2','cat3','sku_size']

# Reordena las columnas del DataFrame
filtered_df = filtered_df[new_order]

In [39]:
filtered_df.head()

,periodo,periodo_fecha,product_id,customer_id,tn,tn_01,tn_md,cust_request_qty,cust_request_tn,cust_request_tn_01,cust_request_tn_md,plan_precios_cuidados,cat1,cat2,cat3,sku_size
4797,201701,2017-01-01,20007,10011,4.90379,0.008943,-0.647465,5,4.90379,0.008883,-0.650233,0,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,900.0
4812,201701,2017-01-01,20007,10005,4.20325,0.007664,-0.661350,6,4.20325,0.007613,-0.663648,0,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,900.0
4823,201701,2017-01-01,20007,10007,2.52973,0.004610,-0.694521,1,2.52973,0.004579,-0.695695,0,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,900.0
4836,201701,2017-01-01,20007,10012,21.19132,0.038671,-0.324629,7,21.19132,0.038413,-0.338329,0,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,900.0
4843,201701,2017-01-01,20007,10003,20.22810,0.036913,-0.343721,22,20.22810,0.036667,-0.356774,0,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,900.0


In [40]:
# Le agregamos las exogenas
print(len(filtered_df))
filtered_df = pd.merge(filtered_df,df_exogenas,on="periodo_fecha",how="left")
print(len(filtered_df))

3744
3744


In [41]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3744 entries, 0 to 3743
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periodo                    3744 non-null   int64         
 1   periodo_fecha              3744 non-null   datetime64[ns]
 2   product_id                 3744 non-null   int64         
 3   customer_id                3744 non-null   int64         
 4   tn                         3744 non-null   float64       
 5   tn_01                      3744 non-null   float64       
 6   tn_md                      3744 non-null   float64       
 7   cust_request_qty           3744 non-null   int64         
 8   cust_request_tn            3744 non-null   float64       
 9   cust_request_tn_01         3744 non-null   float64       
 10  cust_request_tn_md         3744 non-null   float64       
 11  plan_precios_cuidados      3744 non-null   int64         
 12  cat1  

In [42]:
# Exportar el DataFrame a un archivo CSV
filtered_df.to_csv("emp3_sellout_base_period_filtered_customer_product.csv", index=False)